In [1]:
import os
from typing import List, Dict, Any
import pandas as pd




In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print("setup is completed!")

/Users/kaushik003/Documents/projects/RAG/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


setup is completed!


## Understanding Document structure in LangChain

In [4]:
## create a simple doc
doc = Document(
    page_content="This is a sample document for testing purposes.",
    metadata={
        "source": "sample_source",
        "page": 1 ,
        "author": "Test Author",
        "date_created": "2024-06-01"
        
        }
)

print("Document structure")
print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")


Document structure
Content: This is a sample document for testing purposes.
Metadata: {'source': 'sample_source', 'page': 1, 'author': 'Test Author', 'date_created': '2024-06-01'}


In [5]:
type(doc)

langchain_core.documents.base.Document

## Reading Text files (.txt)

In [6]:
## create a simple text file

os.makedirs("data/text_files", exist_ok=True)

In [7]:
sample_texts ={
    "data/text_files/python_intro.txt": """Anyone can learn Python programming.
Python is a versatile language used for web development, data analysis, artificial intelligence, and more.
It has a simple syntax that makes it easy to read and write code.
It has a large community and extensive libraries that support various applications.
It is a great language for beginners and experienced developers alike.""",

    "data/text_files/machine_learning.txt": """ Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from data.
    It involves algorithms that improve their performance as they are exposed to more data over time.
    Common machine learning tasks include classification, regression, clustering, and recommendation.
    Machine learning is widely used in various applications such as image recognition, natural language processing, and predictive analytics.
    Types of machine learning include supervised learning, unsupervised learning, and reinforcement learning.""",


}

for file_path, content in sample_texts.items():
    with open(file_path, "w", encoding = "utf-8" ) as f:
        f.write(content)

print(" Sample text files created.")

 Sample text files created.


## TextLoader - Read single file.

In [10]:

from langchain_community.document_loaders import TextLoader

##Loading single text file
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()
print(f"Loaded {len(documents)} document(s) from python_intro.txt")
print(f"content preview: {documents[0].page_content[:100]}...")
print(f"metadata: {documents[0].metadata}")


Loaded 1 document(s) from python_intro.txt
content preview: Anyone can learn Python programming.
Python is a versatile language used for web development, data a...
metadata: {'source': 'data/text_files/python_intro.txt'}
